In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="COMPAS"
JobNameAbbrev = "CP"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(31,60)),
                    #    "Seed":list([]),                       
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.05],
                       "Type": ["Classification"],
                       "Partition": ["compute-hugemem"],                                                        # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                       "Time": ["2-11:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": ["200G"]}                                                                # [100M, 30000M, 100000M]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Passive Learning

In [3]:
# ### Include Random Forest ###
# RandomForestParameterDictionary = {"Data":[Data],
#                        "Seed":list(range(0,100)),
#                        "TestProportion":[0.2],
#                        "CandidateProportion":[0.8],
#                        "SelectorType":["PassiveLearning"],
#                        "ModelType":["RandomForestClassificationFunction"],
#                        "UniqueErrorsInput": [1],
#                        "n_estimators": [100], 
#                        "regularization": [0.01],
#                        "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
#                        "RashomonThreshold": [0],
#                        "Type": ["Classification"],
#                        "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
#                        "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
#                        "Memory": [1000]}                                                                # [1000, 30000, 100000]

# RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# # NOTE: Comment out chunk to not include random forest simulations. ###
# ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
# ParameterVector = ParameterVector.sort_values("Seed")
# ParameterVector.index = range(0, ParameterVector.shape[0])

# Include/exclude Random Forest Simulations

In [4]:
# ### Include Random Forest ###
# RandomForestParameterDictionary = {"Data":[Data],
#                        "Seed":list(range(0,100)),
#                        "TestProportion":[0.2],
#                        "CandidateProportion":[0.8],
#                        "SelectorType":["TreeEnsembleQBCFunction"],
#                        "ModelType":["RandomForestClassificationFunction"],
#                        "UniqueErrorsInput": [0],
#                        "n_estimators": [100], 
#                        "regularization": [0.01],
#                        "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
#                        "RashomonThreshold": [0],
#                        "Type": ["Classification"],
#                        "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
#                        "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
#                        "Memory": [1000]}                                                                # [1000, 30000, 100000]

# RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# # NOTE: Comment out chunk to not include random forest simulations. ###
# ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
# ParameterVector = ParameterVector.sort_values("Seed")
# ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [5]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI1_", "_PL", regex=True)
    )

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

---

# Only run these ones

In [6]:
# # ### Simulations that failed ###
# FilterText = ["1B7_UA0.02", "1B7_DA0.02", "68B7_UA0.02", "46B7_UA0.02", "52B7_UA0.02", "73B7_UA0.02", "80B7_UA0.02", "57B7_DA0.02", "76B7_DA0.02", "82B7_DA0.02", "87B7_DA0.02", "92B7_DA0.02", "99B7_DA0.02", "4B7_DA0.02", "30B7_DA0.02", "38B7_DA0.02"]

# ### Rerun these simulations ###
# ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector

---

In [7]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,Partition,Time,Memory,JobName,Output
0,COMPAS,31,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,31CP_DA0.05,COMPAS/TreeFarms/Raw/31CP_DA0.05.pkl
1,COMPAS,31,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,31CP_UA0.05,COMPAS/TreeFarms/Raw/31CP_UA0.05.pkl
2,COMPAS,32,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,32CP_DA0.05,COMPAS/TreeFarms/Raw/32CP_DA0.05.pkl
3,COMPAS,32,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,32CP_UA0.05,COMPAS/TreeFarms/Raw/32CP_UA0.05.pkl
4,COMPAS,33,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,33CP_DA0.05,COMPAS/TreeFarms/Raw/33CP_DA0.05.pkl
5,COMPAS,33,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,33CP_UA0.05,COMPAS/TreeFarms/Raw/33CP_UA0.05.pkl
6,COMPAS,34,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,34CP_DA0.05,COMPAS/TreeFarms/Raw/34CP_DA0.05.pkl
7,COMPAS,34,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,34CP_UA0.05,COMPAS/TreeFarms/Raw/34CP_UA0.05.pkl
8,COMPAS,35,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,35CP_DA0.05,COMPAS/TreeFarms/Raw/35CP_DA0.05.pkl
9,COMPAS,35,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Adder,0.05,Classification,compute-hugemem,2-11:59:00,200G,35CP_UA0.05,COMPAS/TreeFarms/Raw/35CP_UA0.05.pkl


# Save Parameter Vector

In [8]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)